In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

In [2]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
#from pyimagesearch.lenet import LeNet
from imutils import paths

Using TensorFlow backend.


In [3]:
# import the necessary packages
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K

In [4]:
class LeNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model
		model = Sequential()
		inputShape = (height, width, depth)

		# if we are using "channels first", update the input shape
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)

		# first set of CONV => RELU => POOL layers
		model.add(Conv2D(64, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# second set of CONV => RELU => POOL layers
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
		# third set of CONV => RELU => POOL layers
		model.add(Conv2D(256, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
              
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(1000))
		model.add(Activation("relu"))
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os

In [6]:
# This is code from the original tutorial for parsing the images
# construct the argument parse and parse the args
#ap = argparse.ArgumentParser()
#ap.add_argument("-d", "--dataset", required =True,
#               help="path to input dataset")
#ap.add_argument("-m", "--model", required=True,
#               help="path to output model")
#ap.add_argument("-p", "--plot", type=str, default="plot.png",
#               help="path to output accuracy/loss plot")
#args = vars(ap.parse_args())

In [23]:
# init the number of epochs to train for, init learning rate and batch size
EPOCHS = 50
INIT_LR = 1e-3
BS = 32

In [24]:
# init the image suffix, yset, and image list
print("[INFO] loading images...")
suffix = '.jpg'
img_list = []
yset = []

[INFO] loading images...


In [25]:
# create labels list and 2 dicts for 2 way mapping
labels = []
# key = label value = number
label_yval = dict()
# key = number value = label
yval_label = dict()

In [26]:
# Testing Cells
csv_file = 'zaslavsk_Cyclops_Cave_Ceramic_Petrography.csv'

In [27]:
# use csv file to grab images/labels
df = pd.read_csv(csv_file)
nameCol = df['#img']
fabricCol = df['Fabric Code']

In [28]:
# add all fabric columns to the y set
for i in range (0,len(fabricCol)):
    labels.append(fabricCol[i])

In [29]:
# grab all unique labels
uni_labels = set(labels)
uni_labels = list(uni_labels)

In [30]:
# assign each label a dict key number
for i in range(0,len(uni_labels)):
    yval_label[i] = uni_labels[i]
    label_yval[uni_labels[i]] = i

In [31]:
label_yval

{'Cyclops Cave 1': 5,
 'Cyclops Cave 10': 3,
 'Cyclops Cave 11': 8,
 'Cyclops Cave 12': 10,
 'Cyclops Cave 2': 7,
 'Cyclops Cave 3': 0,
 'Cyclops Cave 4': 2,
 'Cyclops Cave 5': 6,
 'Cyclops Cave 6': 11,
 'Cyclops Cave 7': 9,
 'Cyclops Cave 8': 1,
 'Cyclops Cave 9': 4}

In [32]:
len(labels)

252

In [33]:
label_yval[labels[0]]

5

In [34]:
# create list of keys associated with their labels
for i in range (0, len(labels)):
    yset.append(label_yval[labels[i]])

In [35]:
len(yset)

252

In [36]:
# remove text and leave fabric cave number for labels and zero index
#for i in range (0,len(yset)):
#    yset[i] = int(re.sub("\D", "", yset[i]))
#    yset[i] = yset[i]-1

In [37]:
# gather images from path created from file names in csv file
for i in range (0,len(nameCol)):
    base_filename = nameCol[i]
    fileName = os.path.join("./Cyclops Cave/images/", base_filename + suffix)
    im = cv2.imread(fileName)
    im = cv2.resize(im, (28,28))
    im = img_to_array(im)
    img_list.append(im)

In [38]:
Y = []
X = []

In [39]:
# Shuffle the data
p = np.random.permutation(len(yset))

In [40]:
for i in range(0,len(yset)):
    Y.append(yset[p[i]])
    X.append(img_list[p[i]])

In [41]:
# split the test and training set 75:25
split = int(len(X)*(.75))
xtrain = X[:split]
xtest = X[split:]
ytrain = Y[:split]
ytest = Y[split:]

In [42]:
# transform to arrays
trainX = np.array(xtrain, dtype="float")/225.0
testX = np.array(xtest, dtype ="float")/225.0

ytrain = np.array(ytrain)
ytest = np.array(ytest)

In [43]:
trainX.shape

(189, 28, 28, 3)

In [44]:
# parsed Y data containers
trainY = []
testY = []

In [45]:
# convert labels from int to vectors
trainY = np_utils.to_categorical(ytrain,12)
testY = np_utils.to_categorical(ytest,12)

In [46]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")

In [47]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=3, classes=12)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
                metrics=["accuracy"])

[INFO] compiling model...


In [48]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

[INFO] training network...
Epoch 1/50
6/5 [====================================] - 3s 536ms/step - loss: 2.0116 - acc: 0.4608 - val_loss: 1.6991 - val_acc: 0.6190
Epoch 2/50
6/5 [====================================] - 2s 402ms/step - loss: 1.8576 - acc: 0.5452 - val_loss: 1.7048 - val_acc: 0.6190
Epoch 3/50
6/5 [====================================] - 3s 424ms/step - loss: 1.7328 - acc: 0.5600 - val_loss: 1.5852 - val_acc: 0.6190
Epoch 4/50
6/5 [====================================] - 2s 393ms/step - loss: 1.6819 - acc: 0.5604 - val_loss: 1.4900 - val_acc: 0.6190
Epoch 5/50
6/5 [====================================] - 2s 344ms/step - loss: 1.5920 - acc: 0.5597 - val_loss: 1.4636 - val_acc: 0.6190
Epoch 6/50
6/5 [====================================] - 2s 343ms/step - loss: 1.5327 - acc: 0.5597 - val_loss: 1.4289 - val_acc: 0.6190
Epoch 7/50
6/5 [====================================] - 2s 349ms/step - loss: 1.4881 - acc: 0.5535 - val_loss: 1.4454 - val_acc: 0.6190
Epoch 8/50
6/5 [=====

In [49]:
# plot the training loss and accuracy 
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Label Prediction")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('figure.png')

In [50]:
# Shows chart in a popup. Close to keep going
chart = cv2.imread('figure.png',1)
cv2.imshow('Results',chart)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [51]:
# Reshape original input data images for predicting
img_check = np.array(img_list, dtype ="float")/225.0

In [52]:
predictionsMade = []

In [53]:
preds = model.predict(img_check)

In [54]:
# Run all data through the prediction model that was created
for i in range (0,len(img_check)):
    predIndex = np.where(preds[i] == np.amax(preds[i]))
    prediction = int(predIndex[0][0])
    predictionsMade.append(prediction)

In [55]:
# Count how many correct predictions were made
correct = 0
for i in range (0,len(predictionsMade)):
    if(predictionsMade[i] == yset[i]):
        correct += 1 

In [56]:
print("Accuracy: " + str(correct/len(yset)))

Accuracy: 0.7619047619047619


In [57]:
# Translate back to original csv label names
finalPred = []
for i in range (0,len(predictionsMade)):
    finalPred.append(yval_label[predictionsMade[i]])

In [58]:
# Append the new column
df['#predictions'] = finalPred

In [61]:
# new file name
new_file =  csv_file[:-4]+'_v1.csv'
df.to_csv(new_file, index=None)